## Members
- Cao Thanh Khiết   19120544

# Pre-processing

In [141]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix


from vncorenlp import VnCoreNLP

In [142]:
vncorenlp_file = "VnCoreNLP/VnCoreNLP-1.1.1.jar"

In [143]:
data_df = pd.read_csv("vfnd/CSV/vn_news_223_tdlfr.csv")
data_df.head()

,text,domain,label
0,Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...,binhluan.biz,1
1,Thủ tướng Nhật cúi đầu xin lỗi vì tinh thần ph...,www.ipick.vn,1
2,Choáng! Cơ trưởng đeo khăn quàng quẩy banh nóc...,tintucqpvn.net,1
3,Chưa bao giờ nhạc Kpop lại dễ hát đến thế!!!\n...,tintucqpvn.net,1
4,"Đại học Hutech sẽ áp dụng cải cách ""Tiếq Việt""...",www.gioitreviet.net,1


In [144]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    223 non-null    object
 1   domain  223 non-null    object
 2   label   223 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 5.4+ KB


##### List of vietnamese stopwords

In [145]:
vn_stopwords = []
with open('data/vietnamese_stopwords.txt', encoding="utf8") as file:
    for line in file.read().splitlines():
        vn_stopwords.append(line.strip())

## EDA (Exploratory Data analysis)

##### Dữ liệu có bị lặp không

In [146]:
data_df.index.duplicated().sum()

0

##### Dữ liệu có null hay không

In [147]:
data_df.isnull().sum()

text      0
domain    0
label     0
dtype: int64

In [148]:
'''
We will extract X_train and Y_train, X_train will contains only column "text", Y_train contains column "label"
'''
X_df = data_df.iloc[:, :-2].values
Y_target = data_df.iloc[:, -1].values

### Loại bỏ một số giá trị không cần thiết
- Loại bỏ stopwords
- Tokenize đoạn văn thành những từ có nghĩa

In [149]:
def preprocessing(X_df):

    for index, test_str in enumerate(X_df):
        # Remove http links
        filtered_str = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", test_str[0])
        # Remove all number in string
        filtered_str = ''.join([i for i in filtered_str if not i.isdigit()])
        # Remove all special characters and punctuation
        filtered_str = re.sub('\W+',' ', filtered_str)
        filtered_str = filtered_str.strip()
        X_df[index][0] = filtered_str.lower()

    X_df = X_df.flatten()


    # Remove stop words
    for index, test_str in enumerate(X_df):
        sw_removed = [w for w in test_str.split(' ') if not w in vn_stopwords]
        X_df[index] = ' '.join(sw_removed)

    return X_df

# Tokenize the sentences into words
def tokennize_text(X_df):
    '''
    return a numpy array of preprocessed text, each item in array is a paragraph after remove stopwords, special characters, ...
    '''
    # annotator = VnCoreNLP(vncorenlp_file, annotators="wseg,pos,ner,parse", max_heap_size='-Xmx2g')
    annotator = VnCoreNLP(vncorenlp_file)
    with annotator:
        for index, filtered_text in enumerate(X_df):
            token_text = annotator.tokenize(filtered_text)[0]
            X_df[index] = ' '.join(token_text)
    
    return X_df

In [150]:
'''
Now, apply preprocessing into my text
'''
X_df = preprocessing(X_df)
X_df = tokennize_text(X_df)

In [151]:
'''
Now, we create a Document Term Matrix (Bag of Words), we will vectorize the normal text
'''
# Use CountVectorizer to convert text data into numerical values
def create_DTM(X_df):
    cv = CountVectorizer(analyzer='word')
    data = cv.fit_transform(X_df).todense()
    return data


'''TF-IDF'''
def vectorize_tfidf(X_df):
    vectorization = TfidfVectorizer()
    return vectorization.fit_transform(X_df).todense()

## Build Model

### Logistic Regression

In [152]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
xv_train = vectorize_tfidf(X_df)
X_train, X_test, y_train, y_test = train_test_split(xv_train, Y_target, test_size=0.2, random_state=0)

In [153]:
LR = LogisticRegression()
LR.fit(X_train, y_train)
acc_test = round(LR.score(X_test, y_test)*100, 2)
print(f"Accuracy on test set: {acc_test}%")

Accuracy on test set: 86.67%


In [154]:
pred_LR = LR.predict(X_test)
pred_LR

array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0], dtype=int64)

In [155]:
cfs_mat = confusion_matrix(y_test, pred_LR)
cfs_mat

array([[25,  2],
       [ 4, 14]], dtype=int64)

### Decision Tree

In [156]:
data = create_DTM(X_df)
X_train, X_test, y_train, y_test = train_test_split(data, Y_target, test_size=0.2, random_state=0)

In [157]:
'''
Build model and predict in test dataset
'''
dtc = DecisionTreeClassifier(criterion='entropy')
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0], dtype=int64)

In [158]:
cfs_mat = confusion_matrix(y_test, y_pred)
cfs_mat

array([[24,  3],
       [10,  8]], dtype=int64)

In [159]:
accuracy = round((cfs_mat[0][0] + cfs_mat[1][1]) / (cfs_mat[0][0] + cfs_mat[0][1] + cfs_mat[1][0] + cfs_mat[1][1])*100, 2)
print(f"Accuracy on test set: {accuracy}%")

Accuracy on test set: 71.11%


### Gradient Booting Classifier

### Deploy